<a href="https://colab.research.google.com/github/davidashirov/uzum_biz_an/blob/main/HW02_uzum_linear_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Практическое задание 2
# Классификация заемщиков линейными моделями

## Студент: Давид Аширов

## Формулировка задания
Данное задание направлено на ознакомление с линейными моделями.

От вас требуется:
1. Загрузить данные
2. Провести первичный анализ данных, изучить имеющиеся признаки
3. Провести обработку данных и обучить простую линейную модель
4. С помощью кросс-валидации + GridSearch подобрать основные параметры для линейной модели. Провести простой отбор признаков
5. Посмотреть как изменение остальных параметров влияет на метрики
6. Оценить качество полученной модели

## 1. Загрузка данных (5 баллов)

Эксперименты будем проводить на [датасете](https://www.kaggle.com/competitions/home-credit-default-risk/overview) по классификации заемщиков на плохих (target = 1: клиент с "payment difficulties") и хороших (target = 0: все остальные). Для экспериментов будем использовать лишь основной файл `application_train.csv`, а также перекодируем таргет в метки -1, 1.

Описание колонок находится в файле `description.csv`.

Для начала мы за вас считаем данные и поделим на обучение, валидацию и тест случайным образом.

In [1]:
!mkdir 'data'
!cd 'data'
!wget 'https://github.com/SergeyMalashenko/MachineLearning_Summer_2023/raw/main/homeworks/10.%20Linear%20Models/data/application_train.zip' -P 'data' -q
!wget 'https://github.com/SergeyMalashenko/MachineLearning_Summer_2023/raw/main/homeworks/10.%20Linear%20Models/data/HomeCredit_columns_description.csv' -P 'data' -q
!unzip -d 'data' 'data/application_train.zip'
!rm 'data/application_train.zip'

Archive:  data/application_train.zip
  inflating: data/application_train.csv  


In [2]:
from pathlib import Path
path = Path('/content/data/application_train.csv')

In [3]:
import numpy as np, pandas as pd

data = pd.read_csv(path)
data.columns = ['_'.join([word.lower() for word in col_name.split(' ') if word != '-']) for col_name in data.columns]

data.target = data.target.map({0: -1, 1: 1})

from IPython.display import display

print('\ntarget value_counts:')
display(data['target'].value_counts(dropna=False))

pd.options.display.max_columns = 100
pd.options.display.max_rows = 150

data.head(5)


target value_counts:


-1    282686
 1     24825
Name: target, dtype: int64

,sk_id_curr,target,name_contract_type,code_gender,flag_own_car,flag_own_realty,cnt_children,amt_income_total,amt_credit,amt_annuity,amt_goods_price,name_type_suite,name_income_type,name_education_type,name_family_status,name_housing_type,region_population_relative,days_birth,days_employed,days_registration,days_id_publish,own_car_age,flag_mobil,flag_emp_phone,flag_work_phone,flag_cont_mobile,flag_phone,flag_email,occupation_type,cnt_fam_members,region_rating_client,region_rating_client_w_city,weekday_appr_process_start,hour_appr_process_start,reg_region_not_live_region,reg_region_not_work_region,live_region_not_work_region,reg_city_not_live_city,reg_city_not_work_city,live_city_not_work_city,organization_type,ext_source_1,ext_source_2,ext_source_3,apartments_avg,basementarea_avg,years_beginexpluatation_avg,years_build_avg,commonarea_avg,elevators_avg,...,apartments_medi,basementarea_medi,years_beginexpluatation_medi,years_build_medi,commonarea_medi,elevators_medi,entrances_medi,floorsmax_medi,floorsmin_medi,landarea_medi,livingapartments_medi,livingarea_medi,nonlivingapartments_medi,nonlivingarea_medi,fondkapremont_mode,housetype_mode,totalarea_mode,wallsmaterial_mode,emergencystate_mode,obs_30_cnt_social_circle,def_30_cnt_social_circle,obs_60_cnt_social_circle,def_60_cnt_social_circle,days_last_phone_change,flag_document_2,flag_document_3,flag_document_4,flag_document_5,flag_document_6,flag_document_7,flag_document_8,flag_document_9,flag_document_10,flag_document_11,flag_document_12,flag_document_13,flag_document_14,flag_document_15,flag_document_16,flag_document_17,flag_document_18,flag_document_19,flag_document_20,flag_document_21,amt_req_credit_bureau_hour,amt_req_credit_bureau_day,amt_req_credit_bureau_week,amt_req_credit_bureau_mon,amt_req_credit_bureau_qrt,amt_req_credit_bureau_year
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,-637,-3648.0,-2120,NaN,1,1,0,1,1,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,Business Entity Type 3,0.083037,0.262949,0.139376,0.0247,0.0369,0.9722,0.6192,0.0143,0.00,...,0.0250,0.0369,0.9722,0.6243,0.0144,0.00,0.0690,0.0833,0.1250,0.0375,0.0205,0.0193,0.0000,0.00,reg oper account,block of flats,0.0149,"Stone, brick",No,2.0,2.0,2.0,2.0,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,-1,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,State servant,Higher education,Married,House / apartment,0.003541,-16765,-1188,-1186.0,-291,NaN,1,1,0,1,1,0,Core staff,2.0,1,1,MONDAY,11,0,0,0,0,0,0,School,0.311267,0.622246,NaN,0.0959,0.0529,0.9851,0.7960,0.0605,0.08,...,0.0968,0.0529,0.9851,0.7987,0.0608,0.08,0.0345,0.2917,0.3333,0.0132,0.0787,0.0558,0.0039,0.01,reg oper account,block of flats,0.0714,Block,No,1.0,0.0,1.0,0.0,-828.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,-1,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010032,-19046,-225,-4260.0,-2531,26.0,1,1,1,1,1,0,Laborers,1.0,2,2,MONDAY,9,0,0,0,0,0,0,Government,NaN,0.555912,0.729567,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,-815.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,-1,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,297000.0,Unaccompanied,Working,Secondary / secondary special,Civil marriage,House / apartment,0.008019,-19005,-3039,-9833.0,-2437,NaN,1,1,0,1,0,0,Laborers,2.0,2,2,WEDNESDAY,17,0,0,0,0,0,0,Business Entity Type 3,NaN,0.650442,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,2.0,0.0,-617.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,-1,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,513000.0,Unaccompanied,Working,Secondary / secondary special,Sing

In [4]:
# TODO
# Разбейте вашу выборку на трейн-тест-валидацию в пропорциях 0.5-0.2-0.3 соответственно

from sklearn.model_selection import train_test_split
X, y = data.drop(['target'],axis=1), data.target

# Строки данных не связаны между собой, поэтому просто делим случайно
seed = 1
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.5, random_state=seed)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.4, random_state=seed)

In [5]:
print(len(X_train))
print(len(X_test))
print(len(X_valid))

153755
61503
92253



## 2. Первичный анализ данных, изучение признаков (5 баллов)

Изучите признаки, которые имеются в датасете.  
Какие из них относятся к каким типам?
Как они коррелируют с целевой переменной?

In [13]:
# TODO
# Проведите первичное изучение признаков

# Используем train сплит
print(f'Dataset has {len(X_train.columns)} columns')
descr = pd.read_csv(path.parent/'HomeCredit_columns_description.csv', encoding='ISO-8859-1')
descr.drop(['Unnamed: 0', 'Table'], axis=1)

Dataset has 121 columns


,Row,Description,Special
0,SK_ID_CURR,ID of loan in our sample,NaN
1,TARGET,Target variable (1 - client with payment diffi...,NaN
2,NAME_CONTRACT_TYPE,Identification if loan is cash or revolving,NaN
3,CODE_GENDER,Gender of the client,NaN
4,FLAG_OWN_CAR,Flag if the client owns a car,NaN
...,...,...,...
214,NUM_INSTALMENT_NUMBER,On which installment we observe payment,NaN
215,DAYS_INSTALMENT,When the installment of previous credit was su...,time only relative to the application
216,DAYS_ENTRY_PAYMENT,When was the installments of previous credit p...,time only relative to the application
217,AMT_INSTALMENT,What was the prescribed installment amount of ...,NaN


In [14]:
descr.Special.value_counts()

normalized                               53
time only relative to the application    19
hashed                                    9
rounded                                   2
normalized                                1
recoded                                   1
grouped                                   1
Name: Special, dtype: int64

In [ ]:
path.parent/'HomeCredit_columns_description.csv'


## 3. Обработка данных, обучение простой линейной модели  (5 баллов)

Изучите признаки, которые имеются в датасете.  
Какие из них относятся к каким типам?
Как они коррелируют с целевой переменной?
Обучите на ваших данных простую [Логистическую регресиию](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)   
Какое качество вам удалось получить?


In [ ]:
# TODO
# Проведите обработку признаков
# Обучение простую логистическую регресиию


## 4. Оценка поведения модели в зависимости от гиперпараметров  (5 баллов)

Попробуйте поперебирать значения гиперпараметров `l2_coef, batch_size`.   
Какая их комбинация выглядит оптимальной?




In [ ]:
# TODO
# Оцените качество модели в зависимости от l2_coef и batch_size


## 5. GridSearchCV. Отбор признаков  (5 баллов)


С помощью GridSearchCV подберите оптимальные параметры `step_alpha, step_beta` для фиксированных значений `l2_coef, batch_size` с предыдущего шага.  
Проведите отбор признаков любым из известных вам методов

In [ ]:
# TODO
# Подберите гиперпараметры step_alpha, step_beta для фиксированных l2_coef, batch_size
# Проведите отбор признаков


## 6. Оценка качества модели  (5 баллов)


Оцените качество модели с помощью известных вам метрик классификации.  
Что ещё в модели можно было бы улучшить?


In [ ]:
# TODO
# Оцените качество модели с помощью известных вам метрик